In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import wandb

import numpy as np
import seaborn as sns

import sys
BASE_PATH = globals()['_dh'][0].parent.absolute()
sys.path.insert(1, str(BASE_PATH))

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from src.models.iterativeModels import learnable_adaptive_iGCN

In [5]:
dataset = Planetoid(root='data/Planetoid', 
                        name="Cora", 
                        transform=NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [6]:
num_features = dataset.num_features
num_classes = dataset.num_classes
model = learnable_adaptive_iGCN(input_dim=num_features,
                                    output_dim=num_classes,
                                    hidden_dim=32,
                                    num_iterations=4,
                                    dropout=0.5)

In [15]:
print(model.train_schedule.tolist())

[0.5, 0.5, 0.5, 0.5]


In [16]:
import torch
a = torch.Tensor([-2, -1, 0, 1, 2])
a.clamp(0,1)

tensor([0., 0., 0., 1., 1.])